In [1]:
import sys
# sys.path.append("D:\thesis\thesis\rllib")
import os
import rllib as rl
import json
import joblib
import numpy as np
import pandas as pd
dataset_train = "./data_v2/symptoms.csv"
# dataset_test = "./data_preprossing/output/test_cleaned.csv"
# symptom_map_file = "./data/basic/symptoms_db.json"
# condition_map_file = "./data/basic/conditions_db.json"
# group_file = "./data/basic/conditions_group.json"
# test = "./data_preprossing/output/data/total_1k.csv"

In [13]:
#parse test cases
# PATIENT,GENDER,RACE,ETHNICITY,AGE_BEGIN,AGE_END,PATHOLOGY,NUM_SYMPTOMS,SYMPTOMS
import pandas as pd

data = pd.read_excel('./data_v2/Test Use Cases.xlsx',index_col=0)

data.to_csv('./data_v2/test.csv',encoding='utf-8')


PermissionError: [Errno 13] Permission denied: './data_v2/test.csv'

In [2]:
# N:6,7,8
# L:9,10,11
#I : 12 13 14
#C: 15 16  17
#E: 18 19 20

#PATIENT,GENDER,RACE,ETHNICITY,AGE_BEGIN,AGE_END,PATHOLOGY,NUM_SYMPTOMS,SYMPTOMS
import csv
patients = [ [ '' for j in range(9)] for i in range(33)]
data = pd.read_csv('./data_v2/test.csv')
with open('./data_v2/test.csv','r') as f:
    reader = csv.reader(f)
    i = 0
    for row in reader:
        if(row[0]!=''): 
            print(row)
            nums_symptoms =1
            if row[4] !='' :nums_symptoms +=1
            if row[5] =='' :nums_symptoms +=1
            patients[i][0] = row[0]
            patients[i][1] = row[2]
            patients[i][2] = 'white'
            patients[i][3] = 'nonhispanic'
            patients[i][4] = row[1]
            patients[i][5] = row[1]
            patients[i][6] = '(Acute) Mastoiditis'
            patients[i][7] = nums_symptoms
            symptoms = []
            for index in range(nums_symptoms):
                symp = row[3+index]
                nature = row[6+index]
                location= row[9+index].split(',')
                if(len(location)>1):
                    location_main = location[0]
                    location = location[1]
                else:
                    location_main = location[0]
                    location = ''
                duration = row[15+index]
                excitation = row[18+index]
                onset = ''
                frequency = ''
                intensity = ''
                symptom = symp+':'+nature+':' + location_main +':' + location +':'+intensity +':'+duration+':' +onset+':'+ excitation+':' +frequency+'29'
                symptoms.append(symptom)
            patients[i][8] = ";".join(symptoms)
            i +=1

            
         

['PATIENT', 'AGE', 'Gender', 'Symptom 1', 'Symptom 2 (if any)', 'Symptom 3(if any)', 'N', 'N2', 'N3', 'L', 'L2', 'L3', 'I', 'I2', 'I3', 'C', 'C2', 'C3', 'E', 'E2', 'E3']
['Case 1', '45', 'M', 'Altered_breathing', 'Fever', '', 'Dyspnea', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['Case 2', '48', 'F', 'Cough', 'Pain', '', 'Mucus', '', '', '', 'Chest', '', '', '', '', '', '', '', '', '', '']
['Case 3', '33', 'F', 'Vomiting', 'Altered_stool', '', '', 'Bloody', '', '', '', '', '', '', '', '', '', '', '', '', '']
['Case 4', '77', 'M', 'Altered_stool', 'Fatigue', 'Bloating', 'Bloody', '', '', '', '', '', '', '', '', '1_-_7_days', '', '', '', '', '']
['Case 5', '89', 'M', 'Fatigue', 'Fever', 'Bloating', '', '', '', '', '', '', '', '', '', '0_-_5_days', '', '', '', '', '']
['Case 7', '24', 'M', 'Pain', 'Nausea', 'Vomiting', 'Throbbing', '', '', 'Head', '', '', '', '', '', '4-24h', '', '', '', '', '']
['Case 8', '11', 'F', 'Pain', 'Vomiting', '', '', '', '', 'Head, Post_aurical_(be

In [3]:
with open("./data_v2/testcases.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['PATIENT','GENDER','RACE','ETHNICITY','AGE_BEGIN','AGE_END','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS'])
    writer.writerows(patients)

In [7]:
def extract_symptom_nlice(filepath):
    use_cols =['GENDER', 'RACE', 'AGE_BEGIN', 'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS'] 
    df = pd.read_csv(filepath)
    symptoms = df['SYMPTOMS']
    symptoms_db = {}
    nature=[]
    location=[]
    location_main=[]
    vas= []
    excitation= []
    frequency= []
    onset=[]
    duration=[]

    count = 0
    count_nlice = 2
    for sym in symptoms:
        # print(sym)
        symptom_list = sym.split(";")
        for _symp_def in symptom_list:
            sym_list = _symp_def.split(":")
            _symptom, _nature, _location_main,_location, _intensity, _duration, _onset, _exciation, _frequency, _ = _symp_def.split(":")
            if _symptom not in symptoms_db:
                symptoms_db[_symptom] = count
                count+=1
            if _nature not in nature:
                nature.append(_nature) 
            if _location_main not in location_main:
                location_main.append(_location_main)         
            if _location not in location:
                location.append(_location)
            if _frequency not in frequency:
                frequency.append(_frequency)
            if _exciation not in excitation:
                excitation.append(_exciation)
            if _intensity not in vas:
                vas.append(_intensity)
            if _onset not in onset:
                onset.append(_onset)
            if _duration not in duration:
                duration.append(_duration)
    return symptoms_db,nature,location_main,location,frequency,excitation,vas,onset,duration
excitation_encoding={}
vas_encoding = {}
nature_encoding = {}
frequency_encoding = {}
duration_encoding = {}
onset_encoding = {}
location_encoding={} 
location_main_encoding={}

In [9]:
all_symtom,nature,location_main,location,frequency,excitation,vas,onset,duration = extract_symptom_nlice(dataset_train)
data_dir = './data_v2/'
count = 1

for item in nature:
    nature_encoding[item] = count
    count += 1
nature_encoding_file = os.path.join(data_dir, "nature_encoding.json")
with open(nature_encoding_file, "w") as fp:
    sorted_nature = sorted(nature_encoding.keys())
    nature_encoding = {code: idx for idx, code in enumerate(sorted_nature)}
    json.dump(nature_encoding, fp, indent=4)

count = 1
for item in location:
    location_encoding[item] = count
    count += 1
location_encoding_file = os.path.join(data_dir, "body-parts-enc.json")
with open(location_encoding_file, "w") as fp:
    sorted_location = sorted(location_encoding.keys())
    location_encoding = {code: idx for idx, code in enumerate(sorted_location)}
    json.dump(location_encoding, fp, indent=4)
count = 1
for item in location_main:
    location_main_encoding[item] = count
    count += 1
location_main_encoding_file = os.path.join(data_dir, "body_main_parts-enc.json")
with open(location_main_encoding_file, "w") as fp:
    sorted_location = sorted(location_main_encoding.keys())
    location_main_encoding = {code: idx for idx, code in enumerate(sorted_location)}
    json.dump(location_main_encoding, fp, indent=4)
count = 1
for item in frequency:
    frequency_encoding[item] = count
    count += 1
frequency_encoding_file = os.path.join(data_dir, "frequency_encoding.json")
with open(frequency_encoding_file, "w") as fp:
    sorted_frequency = sorted(frequency_encoding.keys())
    frequency_encoding = {code: idx for idx, code in enumerate(sorted_frequency)}
    json.dump(frequency_encoding, fp, indent=4)

count = 1
for item in excitation:
    excitation_encoding[item] = count
    count += 1
excitation_encoding_file = os.path.join(data_dir, "excitation_encoding.json")
with open(excitation_encoding_file, "w") as fp:
    sorted_excitation = sorted(excitation_encoding.keys())
    excitation_encoding = {code: idx for idx, code in enumerate(sorted_excitation)}
    json.dump(excitation_encoding, fp, indent=4)

count = 1
for item in vas:
    vas_encoding[item] = count
    count += 1
vas_encoding_file = os.path.join(data_dir, "vas_encoding.json")
with open(vas_encoding_file, "w") as fp:
    sorted_vas = sorted(vas_encoding.keys())
    vas_encoding = {code: idx for idx, code in enumerate(sorted_vas)}
    json.dump(vas_encoding, fp, indent=4)

count = 1
for item in onset:
    onset_encoding[item] = count
    count += 1
onset_encoding_file = os.path.join(data_dir, "onset_encoding.json")
with open(onset_encoding_file, "w") as fp:
    sorted_onset = sorted(onset_encoding.keys())
    onset_encoding = {code: idx for idx, code in enumerate(sorted_onset)}
    json.dump(onset_encoding, fp, indent=4)

count = 1
for item in duration:
    duration_encoding[item] = count
    count += 1
duration_encoding_file = os.path.join(data_dir, "duration_encoding.json")
with open(duration_encoding_file, "w") as fp:
    sorted_duration = sorted(duration_encoding.keys())
    duration_encoding = {code: idx for idx, code in enumerate(sorted_duration)}
    json.dump(duration_encoding, fp, indent=4)

In [ ]:
output_path = "./data_preprossing/output/data/"
data1k = "./data_preprossing/output/data/total_1k.csv"
classlabel = [24, 29, 49, 38, 5, 3, 43, 48, 41, 37, 12, 31, 9, 42, 35, 28, 8, 14, 36, 4, 40, 19, 20, 21, 39, 53, 10, 25, 0, 44, 2, 17, 45]
symptom_columns = ['Index','GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
symptom_columns = ['GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
df = pd.read_csv(dataset_train,names=symptom_columns)
df.index.name = "Index"
df['SYMPTOMS'] = df.SYMPTOMS.apply(lambda symptom: symptom if len(symptom.split(";")) >=3 else np.NAN)
df['SYMPTOMS'].replace('', np.nan, inplace=True)
df.dropna(subset=['NUM_SYMPTOMS'],inplace=True)
df.dropna(subset=['SYMPTOMS'],inplace=True)
df = df.drop_duplicates()
label = "total_1k_min3"
output_file = os.path.join(output_path, "%s.csv" % label)
df.to_csv(output_file)

In [14]:
group = {}
#1 C - Pulmonary


group["1"] = ["viral respiratory infection (COVID-19 most likely)","viral respiratory infection (Influenza most likely)","Pneumonia (Pneumococcal most likely)","Pneumonia (Adenovirus most likely)"
            ,"COPD","Asthma","Pneumothorax","Pulmonary edema (among young)","Pulmonary edema (among old)","Pulmonary edema (among oldest)"]
#C - Gastrointestinal
group["2"] = ["Bacterial (Gastro)enteritis (Salmonella infection most likely)","Bacterial (Gastro)enteritis (Yersinia infection most likely)"
            ,"Viral (Gastro)enteritis (Norovirus infection most likely)","Giardiasis","Appendicitis","Functional constipation",
            "Acute pancreatitis","IBS (Constipation type)","Atrophic gastritis"]

#NC - Rheumatology
group["3"] = ["Gout","Rheumatoid arthritis (F)","Rheumatoid arthritis (M)","Osteoarthritis"]

#NC - Neurological
group["4"] = ["Migraine","Tension type headache","Subarachnoidal bleeding","Bacterial meningitis","Viral meningitis (Enterovirus)","Viral meningitis (Varicella zoster virus)"]


#NC - Neuro-orthopaedic
group["5"] = ["Hernia Nuclei Pulposi (Radicular syndrome)","Lumbago (Muscle strain)","Cauda equina (Radicular Syndrome)","Lumbar spinal stenosis (Radicular syndrome)"
                ,"Carpal tunnel syndrome"]

#C - Oropharyngeal
group["6"] = ["Otitis media acuta","(Acute) Mastoiditis","Otitis externa"]

#STDs
group["7"] = ["Gonorrhea (F)","Genital herpes (M)","Genital herpes (F)"]

#NC - Oncological
group["8"] = ["Lung cancer (NSCLC)","Colon cancer","Testicular cancer","Breast cancer","Ovarian cancer","Oesophageal carcinoma","Bladder cancer"]

#NC - Deficiencies
group["9"] = ["Iron deficiency anaemia","Vitamin D deficiency","Vitamin B12 deficiency","Wernicke-Korsakoff complex"]

#C - Urological
group["10"] = ["Female cystitis","Acute pyelonefritis","Upper urinary tract infection","Lower urinary tract infection"]



In [15]:
output_path = "./data_v2/group/"

group_id = [1,2,3,4,5,6,7,8,9,10]
symptom_columns = ['Index','GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
df = pd.read_csv(dataset_train,sep = "\t")
for i in group_id:
    syms = group[str(i)]
    # symptom_columns = ['GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
    df = pd.read_csv(dataset_train)
    df = df.loc[df['PATHOLOGY'].isin(syms)]
    # df.index.name = "Index"
#     df = df.loc[df['NUM_SYMPTOMS']>=2]
    df['NUM_SYMPTOMS'] = df.NUM_SYMPTOMS.apply(lambda n: int(n) if int(n) >=2 else np.NAN)
    df['NUM_SYMPTOMS'].replace('', np.nan, inplace=True)
    df['SYMPTOMS'] = df.SYMPTOMS.apply(lambda symptom: symptom if len(symptom.split(";")) >=2 else np.NAN)
    df['SYMPTOMS'].replace('', np.nan, inplace=True)
    df.dropna(subset=['NUM_SYMPTOMS'],inplace=True)
    df.dropna(subset=['SYMPTOMS'],inplace=True)
    df = df.drop_duplicates()
    if(len(df)>=2):
        output_file = os.path.join(output_path, "group"+str(i),"%s.csv" % str(i))
        df.to_csv(output_file)

In [16]:
output_path = "./data_v2/data/"
## extract train and test dataset
group_id = [1,2,3,4,5,6,7,8,9,10]
symptom_columns = ['GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
# df = pd.read_csv(dataset_train,sep = "\t")
for i in group_id:
    for condition in group[str(i)]:
        df = pd.read_csv(dataset_train)
        df = df.loc[df['PATHOLOGY'] == condition]
        df['NUM_SYMPTOMS'] = df.NUM_SYMPTOMS.apply(lambda n: int(n) if int(n) >=2 else np.NAN)
        df['NUM_SYMPTOMS'].replace('', np.nan, inplace=True)
        df['SYMPTOMS'] = df.SYMPTOMS.apply(lambda symptom: symptom if len(symptom.split(";")) >=2 else np.NAN)
        df['SYMPTOMS'].replace('', np.nan, inplace=True)
        df.dropna(subset=['NUM_SYMPTOMS'],inplace=True)
        df.dropna(subset=['SYMPTOMS'],inplace=True)
        df = df.drop_duplicates()
        if(len(df)>=2):
            output_file = os.path.join(output_path,"%s.csv" % str(condition))
            df.to_csv(output_file,index=False)

In [5]:
output_path = "./data_v2/group/"
## extract train and test dataset
group_id = [1,2,3,4,5,6,7,8,9,10]
symptom_columns = ['GENDER','RACE','AGE_BEGIN','PATHOLOGY','NUM_SYMPTOMS','SYMPTOMS']
# df = pd.read_csv(dataset_train,sep = "\t")
for i in group_id:
        file = output_path + "group"+str(i)+"/"+str(i)+".csv"
        df = pd.read_csv(file)
        n = df['NUM_SYMPTOMS'].mean()
        print(n)



     

3.330171722359105
3.5356214917862148
4.42300216444052
3.970497758669533
3.2051702330115113
3.9791949913151186
4.552267268360143
3.172100039632394
3.531584565008955
2.794129889535194


In [17]:
import os, glob
import pandas as pd

path = "./data_v2/data"
output = "./data_v2"

files = os.listdir(path)  #获取文件夹下所有文件名
print(files)
df1 = pd.read_csv(path + '/' + files[0])  #读取首个csv文件，保存到df1中
df1 = df1.iloc[0:1000]
for file in files[1:]:     
  print(file)
  df2 = pd.read_csv(path +'/' +  file)  #打开csv文件，注意编码问题，保存到df2中
  df2 = df2.iloc[0:1000] # first five rows of dataframe
  df1 = pd.concat([df1,df2],axis=0,ignore_index=True)  #将df2数据与df1合并

df1 = df1.drop_duplicates()   #去重
# df1 = df1.reset_index(drop=True) #重新生成index
df1.to_csv(output + '/' + 'total_1k.csv',index=False) #将结果保存为新的csv文件

['(Acute) Mastoiditis.csv', 'Acute pancreatitis.csv', 'Acute pyelonefritis.csv', 'Appendicitis.csv', 'Asthma.csv', 'Atrophic gastritis.csv', 'Bacterial (Gastro)enteritis (Salmonella infection most likely).csv', 'Bacterial (Gastro)enteritis (Yersinia infection most likely).csv', 'Bacterial meningitis.csv', 'Bladder cancer.csv', 'Breast cancer.csv', 'Carpal tunnel syndrome.csv', 'Cauda equina (Radicular Syndrome).csv', 'Colon cancer.csv', 'COPD.csv', 'Female cystitis.csv', 'Functional constipation.csv', 'Genital herpes (F).csv', 'Genital herpes (M).csv', 'Giardiasis.csv', 'Gonorrhea (F).csv', 'Gout.csv', 'Hernia Nuclei Pulposi (Radicular syndrome).csv', 'IBS (Constipation type).csv', 'Iron deficiency anaemia.csv', 'Lower urinary tract infection.csv', 'Lumbago (Muscle strain).csv', 'Lumbar spinal stenosis (Radicular syndrome).csv', 'Lung cancer (NSCLC).csv', 'Migraine.csv', 'Oesophageal carcinoma.csv', 'Osteoarthritis.csv', 'Otitis externa.csv', 'Otitis media acuta.csv', 'Ovarian cancer.c

In [18]:
import os, glob
import pandas as pd

path = "./data_v2/data"
output = "./data_v2"

files = os.listdir(path)  #获取文件夹下所有文件名
print(files)
df1 = pd.read_csv(path + '/' + files[0])  #读取首个csv文件，保存到df1中
df1 = df1.iloc[0:200]
for file in files[1:]:     
  print(file)
  df2 = pd.read_csv(path +'/' +  file)  #打开csv文件，注意编码问题，保存到df2中
  df2 = df2.iloc[0:200] # first five rows of dataframe
  df1 = pd.concat([df1,df2],axis=0,ignore_index=True)  #将df2数据与df1合并

df1 = df1.drop_duplicates()   #去重
# df1 = df1.reset_index(drop=True) #重新生成index
df1.to_csv(output + '/' + 'total_200.csv',index=False) #将结果保存为新的csv文件

['(Acute) Mastoiditis.csv', 'Acute pancreatitis.csv', 'Acute pyelonefritis.csv', 'Appendicitis.csv', 'Asthma.csv', 'Atrophic gastritis.csv', 'Bacterial (Gastro)enteritis (Salmonella infection most likely).csv', 'Bacterial (Gastro)enteritis (Yersinia infection most likely).csv', 'Bacterial meningitis.csv', 'Bladder cancer.csv', 'Breast cancer.csv', 'Carpal tunnel syndrome.csv', 'Cauda equina (Radicular Syndrome).csv', 'Colon cancer.csv', 'COPD.csv', 'Female cystitis.csv', 'Functional constipation.csv', 'Genital herpes (F).csv', 'Genital herpes (M).csv', 'Giardiasis.csv', 'Gonorrhea (F).csv', 'Gout.csv', 'Hernia Nuclei Pulposi (Radicular syndrome).csv', 'IBS (Constipation type).csv', 'Iron deficiency anaemia.csv', 'Lower urinary tract infection.csv', 'Lumbago (Muscle strain).csv', 'Lumbar spinal stenosis (Radicular syndrome).csv', 'Lung cancer (NSCLC).csv', 'Migraine.csv', 'Oesophageal carcinoma.csv', 'Osteoarthritis.csv', 'Otitis externa.csv', 'Otitis media acuta.csv', 'Ovarian cancer.c

In [10]:
output_path = "./data_v2/group/"
def extract_symptom_bygroup(filepath):
    # use_cols =['GENDER', 'RACE', 'AGE_BEGIN', 'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS'] 
    df = pd.read_csv(filepath)
    # with open(symptom_db) as fp:
    #     symptoms_mp = json.load(fp)
    symptoms = df['SYMPTOMS']
    condition = df['PATHOLOGY']
    conditions_db = {}
    count1= 0
    for con in condition:
        if con not in conditions_db:
            conditions_db[con] = count1
            count1+=1

    symptoms_db = {}
    count = 0
    for sym in symptoms:
        # print(sym)
        symptom_list = sym.split(";")
        for _symp_def in symptom_list:
            sym_list = _symp_def.split(":")
            _symptom,_nature, _location_main,_location, _intensity, _duration, _onset, _exciation, _frequency, _ = _symp_def.split(":")
            if(_symptom == "Alterred_stool"):
                _symptom ="Altered_stool"
            if(_symptom == "Nausea_"):
                _symptom="Nausea"
            if(_symptom == "Pain_relief_"):
                _symptom= "Pain_relief"
            if(_symptom == "Vomitting"):
                _symptom== "Vomiting"
            if(_symptom == "Incontinence_"):
                _symptom="Incontinence"
            if _symptom not in symptoms_db:
                symptoms_db[_symptom] = count
                count+=1
    return symptoms_db,conditions_db

for id in group_id:
    file = output_path+"group"+str(id)+"/"+str(id)+".csv"
    all_symtom,all_condition = extract_symptom_bygroup(file)
    all_symtom_file = output_path+"group"+str(id)+"/"+"symptom"+str(id)+".json"
    with open(all_symtom_file, "w") as fp:
        sorted_symptoms = sorted(all_symtom.keys())
        symptom_map = {code: idx for idx, code in enumerate(sorted_symptoms)}
        json.dump(all_symtom, fp, indent=4)

    conditon_file = output_path+"group"+str(id)+"/"+"condition"+str(id)+".json"
    with open(conditon_file, "w") as fp:
        sorted_condition = sorted(all_condition.keys())
        condition_map = {code: idx for idx, code in enumerate(sorted_condition)}
        json.dump(all_condition, fp, indent=4) 